In [65]:
from tqdm import tqdm
import networkx as nx
from copy import deepcopy as cp
import io
import os
import numpy as np
import requests
import json

In [66]:
url = 'https://kb-isand.ipu.ru/grapher/produce_classificator_subtree' # это метод который уже берет из бд
response = requests.post(url, json = {'root_ids' : [1, 211, 1540, 2223], 'return_variants' : True})
result = json.loads(response.text)

KeyboardInterrupt: 

In [ ]:
terms = {elm['name'] : [elm['id'] , elm["id_parent"]] for elm in result['3'] if elm['id'] != 0}
subfactors = {elm['id'] : [elm['name'] , elm["id_parent"]] for elm in result['2'] if elm['id'] != 0}
factors = {elm['id'] : [elm['name'] , elm["id_parent"]] for elm in result['1'] if elm['id'] != 0}
metafactors = {elm['id'] : [elm['name'] , elm["id_parent"]] for elm in result['0'] if elm['id'] != 0}

In [ ]:
metafactors

{1: ['общенаучная проблематика', -1],
 211: ['предметная область', -1],
 1540: ['сфера применения', -1],
 2223: ['математический аппарат', -1]}

In [73]:
def find_term_similarity(term_1, term_2, term_match_result = 2, subfactor_match_result = 1, factor_match_result = 0.7, metafactor_match_result = 0.5):
    if term_1 == term_2:
        return term_match_result
    
    term_1_parent = terms[term_1][1]
    try:
        term_1_g_parent = subfactors[term_1_parent][1]
    except KeyError:
        term_1_g_parent = term_1_parent
    term_1_g_g_parent = factors[term_1_g_parent][1]
    
    term_2_parent = terms[term_1][1]
    try:
        term_2_g_parent = subfactors[term_2_parent][1]
    except KeyError:
        term_2_g_parent = term_2_parent
    term_2_g_g_parent = factors[term_2_g_parent][1]
    
    if term_1_parent == term_2_parent: #у терминов одинаковый предок - подфактор
        # try:
            # print(f"subfactor match: {subfactors[term_2_parent]}")
        # except KeyError:
            # print(f"factor match: {factors[term_2_parent]}")

        if term_2_parent == 2:
            # print(f"common_terms: {factors[term_2_parent]}")
            return metafactor_match_result

        return subfactor_match_result
    
    if term_1_g_parent == term_2_g_parent: # у терминов общий предок - фактор
        # print(f"factor match: {factors[term_2_g_parent]}")
        return factor_match_result
    
    if term_1_g_g_parent == term_2_g_g_parent: # у терминов общий предок - метафактор
        # print(f"metafactor match: {metafactors[term_2_g_g_parent]}")
        return metafactor_match_result
    else:
        return 0
    


In [74]:
import stanza
from nltk.tokenize import sent_tokenize
parser = stanza.Pipeline(lang='ru', processors='tokenize,pos,lemma')

2024-09-19 12:53:37 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2024-09-19 12:53:38 INFO: Downloaded file to C:\Users\User\stanza_resources\resources.json
2024-09-19 12:53:38 INFO: Loading these models for language: ru (Russian):
| Processor | Package            |
----------------------------------
| tokenize  | syntagrus          |
| pos       | syntagrus_charlm   |
| lemma     | syntagrus_nocharlm |

2024-09-19 12:53:38 INFO: Using device: cpu
2024-09-19 12:53:38 INFO: Loading: tokenize
2024-09-19 12:53:38 INFO: Loading: pos
2024-09-19 12:53:38 INFO: Loading: lemma
c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\stanza\models\lemma\trainer.py:236: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to cons

In [76]:
def cnt(text, exmp):
    count_er=0
    start_index=0
    for i in range(len(text)):
        j = text.find(exmp,start_index)
        if(j!=-1):
            start_index = j+1
            count_er+=1
    return count_er

def find_terms_in_sentence(text):
    # parsed_text = parser(text)
    # normalized_text = " ".join([elm.lemma for elm in parsed_text.sentences[0].words])
    # print(normalized_text)
    term_list = {}
    for term in terms:
        term_cnt = cnt(text, " " + term + " ")
        if term_cnt != 0:
            # try:
            #     # print(factors[subfactors[terms[term][1]][1]][1])
            #     if factors[subfactors[terms[term][1]][1]][1] == 1:
            #         continue
            # except KeyError:
            #     # print(factors[terms[term][1]][1])
            #     if factors[terms[term][1]][1] == 1:
            #         continue

            term_list[term] = term_cnt
    return term_list   


In [ ]:

find_terms_in_sentence("Характерной особенностью решения прикладных задач исследования реальных систем является невозможность получения значений всех параметров по всем объектам во все моменты времени что приводит к пропускам в данных")



{'объект': 1, 'исследование': 1, 'система': 1, 'решение': 1, 'задача': 1}

In [ ]:
def load_testing_sample(fname, normalize = False):
    with io.open(fname, "r", encoding='utf-8') as f:
        text_ = f.read()
    sentences = text_.split("\n")

    sentences_without_breaks = []
    true_segment_breaks = []
    prev_sent = None
    for sent in sentences:
        if sent.strip() != "":
            if sent != "--- SEGMENT BRAKE ---":
                if normalize:
                    sent_norm_text = ""
                    try:
                        doc = parser(sent).sentences[0]
                        for elm in doc.words:
                            sent_norm_text += elm.lemma + ' '
                    except IndexError as err:# это встретилось пустое предложение
                        raise err
                else:
                    sent_norm_text = sent
                sentences_without_breaks.append(sent_norm_text)
        
                if prev_sent is not None:
                    if prev_sent == "--- SEGMENT BRAKE ---":
                        true_segment_breaks.append(True)
                    else:
                        true_segment_breaks.append(False)
            prev_sent = sent
    true_segment_breaks = np.array(true_segment_breaks)
    return true_segment_breaks, sentences_without_breaks

In [ ]:
texts_directory = "segmentation_dataset"

all_true_breaks = []
all_sentences_without_breaks = []
for fname in tqdm(os.listdir(texts_directory)):
    path = texts_directory + "/" + fname
    true_breaks, sentences_without_breaks = load_testing_sample(path, normalize = True)
    all_true_breaks.append(true_breaks)
    all_sentences_without_breaks.append(sentences_without_breaks)

100%|██████████| 64/64 [06:30<00:00,  6.10s/it]


In [95]:
true_segment_breaks, sentences = all_true_breaks[0], all_sentences_without_breaks[0]

In [96]:
sims = []
for i in range(len(sentences) - 1):
    sent_1_terms = find_terms_in_sentence(sentences[i])
    sent_2_terms = find_terms_in_sentence(sentences[i+1])
    print(sentences[i])
    print(sent_1_terms)
    print(f"break: {true_segment_breaks[i]}\t\t",end='')

    sent_sim = 0
    for term_1 in sent_1_terms:
        for term_2 in sent_2_terms:
            sent_sim += find_term_similarity(term_1,term_2, term_match_result=2, subfactor_match_result=1, factor_match_result=1,metafactor_match_result=0.01)
    sent_sim /= (len(sent_2_terms)* len(sent_1_terms)) if (len(sent_2_terms)* len(sent_1_terms)) != 0 else 1

    print(f"sim: {sent_sim}")
    sims.append(sent_sim)

один из задача данный работа посвятить вывод уравнение движение КМР в режим управление конфигурация манипулятор с учет специфика использовать тип электромеханический привод и подвижность несущий корпус 
{'управление': 1, 'работа': 1, 'задача': 1, 'манипулятор': 1}
break: False		sim: 0.42333333333333334
решать задача построение математический модель плоский движение свободнолетающий космический манипуляционный робот предназначить для технический обслуживание поверхность пилотировать орбитальный станция 
{'модель': 1, 'задача': 1, 'космический манипуляционный робот': 1}
break: False		sim: 0.34
далее быть предполагать что для каждый координата существовать компонент вектор тонна е тонна и 
{'компонент': 1}
break: False		sim: 0.01
в постановочный часть этот задача быть изложить в 
{'задача': 1, 'часть': 1}
break: True		sim: 0.01
это позволять общество быть более бдительный и не попадаться на уловка злоумышленник 
{'общество': 1}
break: False		sim: 0.0
уже много компания ориентировать на ау

In [77]:
all_sims = []
for sentences in tqdm(all_sentences_without_breaks):
    sims = []
    for i in range(len(sentences) - 1):
        sent_1_terms = find_terms_in_sentence(sentences[i])
        sent_2_terms = find_terms_in_sentence(sentences[i+1])
        # print(sent_1_terms)
        # print(f"break: {true_segment_breaks[i]}\t\t",end='')

        sent_sim = 0
        for term_1 in sent_1_terms:
            for term_2 in sent_2_terms:
                sent_sim += find_term_similarity(term_1,term_2, term_match_result=2, subfactor_match_result=1, factor_match_result=1,metafactor_match_result=0.01)
        sent_sim /= (len(sent_2_terms)* len(sent_1_terms)) if (len(sent_2_terms)* len(sent_1_terms)) != 0 else 1

        # print(f"sim: {sent_sim}")
        sims.append(sent_sim)
    all_sims.append(sims)

100%|██████████| 64/64 [13:23<00:00, 12.55s/it]


In [97]:
for i in range(len(sims)):
    print(f"{sims[i]}\t\t\t{true_segment_breaks[i]}")

0.42333333333333334			False
0.34			False
0.01			False
0.01			True
0.0			False
0.0			False
0.01			False
0.4079999999999998			False
0.20800000000000002			True
0.01			False
0.23111111111111107			False
0.0			False
0.0			True
0.010000000000000002			False
0.0			False
0.0			False
0.01			True
0.01			False
0.01			False
0.3416666666666665			False
0.34			True
0.0			False
0.0			False
0.10949999999999993			False
0.3406666666666665			True
0.2753333333333332			False
0.0			False
0.0			False
0.4079999999999999			False
0.009999999999999998			True
0.01			False
0.17583333333333315			False
0.12055555555555553			False
0.0			True
0.0			False
0.01			False
0.01			False
0.01			True
0.3399999999999999			False
0.2587499999999998			False
0.5062499999999999			False
0.5074999999999998			False
0.010000000000000005			True
0.1094999999999999			False
0.0			False
0.0			False
0.05738095238095235			False
0.06527777777777774			True
0.5066666666666666			False
0.01			False
0.01			False
1.005			True
0.01			False
1.0			False
0.

In [98]:
cutoff = np.array(sims).mean() + np.array(sims).std()

In [99]:
from nltk import windowdiff
from nltk import pk

def predict_breaks(gaps, segment_break_cutoff):
    segment_breaks = []
    for elm in range(len(gaps)):
        if gaps[elm] < segment_break_cutoff: # должно же быть > ?!
            segment_breaks.append(True)
        else:
            segment_breaks.append(False)
    segment_breaks = np.array(segment_breaks)
    return segment_breaks

def find_matching_for_testing_file(sentences_without_breaks, true_segment_breaks, gaps, cutoff, windsize = 3):
    predicted_segment_breaks = predict_breaks(gaps, cutoff)
    t_seg = ''.join(str(int(sb)) for sb in true_segment_breaks)
    seg = ''.join(str(int(sb)) for sb in predicted_segment_breaks)
    
    wnd = windowdiff(''.join(str(int(sb)) for sb in true_segment_breaks), 
                     ''.join(str(int(sb)) for sb in predicted_segment_breaks), windsize)
    pk_ = pk(''.join(str(int(sb)) for sb in true_segment_breaks), 
             ''.join(str(int(sb)) for sb in predicted_segment_breaks), windsize)
    
    return wnd, pk_, t_seg, seg

In [100]:
wd_s = []
pk_s = []
tsegs = []
segs = []
for i in range(len(all_true_breaks)):
    gaps = all_sims[i]
    true_segment_breaks = all_true_breaks[i]
    
    wd_, pk_, tseg, seg = find_matching_for_testing_file(sentences_without_breaks, 
                                              true_segment_breaks, 
                                              gaps, 
                                              cutoff = np.array(gaps).mean() + np.array(gaps).std())
            
    wd_s = np.array(wd_)
    pk_s = np.array(pk_)
    tsegs.append(tseg)
    segs.append(seg)

wd_s_mean = wd_s.mean()
pk_s_mean = pk_s.mean()

In [101]:
print(wd_s_mean)
print(pk_s_mean)

0.9857142857142858
0.35714285714285715


In [102]:
for i in range(len(segs)):
    print(tsegs[i])
    print(segs[i])
    print()

000100001000100010001000100001000100010000100001000100010000100010000
001111101111111111100111011101111111110100111111011010111111111111111

00001000010001000010000100001000010000100001000010000100001000100010000
11101110111110111111111111111111111111111111101111110111011111101111110

0001000010000100010000100010001000010000100010001000011000100010001000
1111111111110110111111011110111111110111111111101111111111111111111111

000100001000100001100001000100001000010001000100010001000010001000010000
111111111100111111111001111111011111101111110001001111111111111111111111

000010001000100010001000010000100010000100010001000010001000100010000
111111111011011011111101111111111111111111101111111111111111011111111

000010000100001000100001000100001000100001000100001000010000100010000
011111111111111110110011111111101111110101111111111100011111111111111

0000100010001000010001000100001000010001000100010000100001000100010000
1111111111111010011111101110111111111111111011111111111110111111111111


In [ ]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)

segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

text = 'Разрабатываемая система обеспечивает к примеру решение задач формирования системы образовательных услуг региона округа района т е задач требующих при решении географической привязки к местности.'
doc = Doc(text)
doc.segment(segmenter)

doc.tag_morph(morph_tagger)
doc.parse_syntax(syntax_parser)
for token in doc.tokens:
#     print(token)
     print(f"{token.pos} \t {token.rel} \t\t\t {token.text} \t\t\t {doc.tokens[int(token.head_id[2:]) -1].text}")


ADJ 	 amod 			 Разрабатываемая 			 система
NOUN 	 nsubj 			 система 			 обеспечивает
VERB 	 root 			 обеспечивает 			 .
ADP 	 case 			 к 			 примеру
NOUN 	 obl 			 примеру 			 обеспечивает
NOUN 	 obj 			 решение 			 обеспечивает
NOUN 	 nmod 			 задач 			 решение
NOUN 	 nmod 			 формирования 			 задач
NOUN 	 nmod 			 системы 			 формирования
ADJ 	 amod 			 образовательных 			 услуг
NOUN 	 nmod 			 услуг 			 системы
NOUN 	 nmod 			 региона 			 системы
NOUN 	 nmod 			 округа 			 системы
NOUN 	 nmod 			 района 			 системы
ADP 	 appos 			 т 			 системы
X 	 fixed 			 е 			 т
NOUN 	 nmod 			 задач 			 решение
VERB 	 nmod 			 требующих 			 формирования
ADP 	 case 			 при 			 решении
NOUN 	 obl 			 решении 			 обеспечивает
ADJ 	 amod 			 географической 			 привязки
NOUN 	 nmod 			 привязки 			 решении
ADP 	 case 			 к 			 местности
NOUN 	 nmod 			 местности 			 привязки
PUNCT 	 punct 			 . 			 обеспечивает


In [ ]:
!python -m spacy download ru_core_news_sm

     ---------------------------------------- 0.0/15.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/15.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/15.3 MB ? eta -:--:--
     --------------------------------------- 0.0/15.3 MB 279.3 kB/s eta 0:00:55
     --------------------------------------- 0.1/15.3 MB 476.3 kB/s eta 0:00:32
     --------------------------------------- 0.1/15.3 MB 652.5 kB/s eta 0:00:24
      -------------------------------------- 0.2/15.3 MB 846.9 kB/s eta 0:00:18
      -------------------------------------- 0.3/15.3 MB 883.3 kB/s eta 0:00:17
     - -------------------------------------- 0.4/15.3 MB 1.1 MB/s eta 0:00:14
     - -------------------------------------- 0.4/15.3 MB 1.0 MB/s eta 0:00:15
     - -------------------------------------- 0.5/15.3 MB 1.0 MB/s eta 0:00:15
     - -------------------------------------- 0.7/15.3 MB 1.3 MB/s eta 0:00:12
     - -------------------------------------- 0.7/15.3 MB 1.3 MB/s e


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import spacy
nlp = spacy.load("ru_core_news_sm")
import ru_core_news_sm
nlp = ru_core_news_sm.load()
doc = nlp('Разрабатываемая система обеспечивает к примеру решение задач формирования системы образовательных услуг региона округа района т е задач требующих при решении географической привязки к местности.')
for token in doc:
    print(f"{token.pos_} \t {token.dep_} \t\t\t {token.lemma_} \t\t\t {token.head} ")
    

VERB 	 amod 			 разрабатывать 			 система 
NOUN 	 nsubj 			 система 			 обеспечивает 
VERB 	 ROOT 			 обеспечивать 			 обеспечивает 
ADP 	 case 			 к 			 примеру 
NOUN 	 obl 			 пример 			 обеспечивает 
NOUN 	 obj 			 решение 			 обеспечивает 
NOUN 	 nmod 			 задача 			 решение 
NOUN 	 nmod 			 формирование 			 задач 
NOUN 	 nmod 			 система 			 формирования 
ADJ 	 amod 			 образовательный 			 услуг 
NOUN 	 nmod 			 услуга 			 системы 
NOUN 	 nmod 			 регион 			 системы 
NOUN 	 nmod 			 округ 			 формирования 
NOUN 	 nmod 			 район 			 округа 
PROPN 	 appos 			 т 			 района 
NOUN 	 fixed 			 е 			 т 
NOUN 	 nmod 			 задача 			 формирования 
VERB 	 nmod 			 требовать 			 задач 
ADP 	 case 			 при 			 решении 
NOUN 	 obl 			 решение 			 требующих 
ADJ 	 amod 			 географический 			 привязки 
NOUN 	 nmod 			 привязка 			 решении 
ADP 	 case 			 к 			 местности 
NOUN 	 nmod 			 местность 			 привязки 
PUNCT 	 punct 			 . 			 обеспечивает 


In [ ]:
import stanza
from nltk.tokenize import sent_tokenize
stanza_parser = stanza.Pipeline(lang='ru', processors='tokenize,pos,lemma,ner,depparse')

c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-08-29 14:11:31 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2024-08-29 14:11:32 INFO: Downloaded file to C:\Users\User\stanza_resources\resources.json
2024-08-29 14:11:33 INFO: Loading these models for language: ru (Russian):
| Processor | Package            |
----------------------------------
| tokenize  | syntagrus          |
| pos       | syntagrus_charlm   |
| lemma     | syntagrus_nocharlm |
| depparse  | syntagrus_charlm   |
| ner       | wikiner            |

2024-08-29 14:11:33 INFO: Using device: cpu
2024-08-29 14:11:33 INFO: Loading: tokenize
c:\Users\

In [ ]:
doc = stanza_parser('\
    Разрабатываемая система обеспечивает к примеру решение задач формирования системы образовательных услуг региона округа района т е задач требующих при решении географической привязки к местности')
for token in doc.sentences[0].words:
    #print(token)
    print(f'{token.upos} \t {token.deprel} \t\t\t {token.lemma} \t\t\t {doc.sentences[0].words[int(token.head) - 1].text}')
    for a in dir(token):
        if a[0] != "_":
            print(a)

    break
    
    

VERB 	 amod 			 разрабатывать 			 система
add_property
coref_chains
deprel
deps
end_char
feats
head
id
lemma
manual_expansion
misc
parent
pos
pretty_print
sent
start_char
text
to_conll_text
to_dict
upos
xpos


ADJ 	 amod 			 Разрабатываемая 			 система
NOUN 	 nsubj 			 система 			 обеспечивает
VERB 	 root 			 обеспечивает 			 .
ADP 	 case 			 к 			 примеру
NOUN 	 obl 			 примеру 			 обеспечивает
NOUN 	 obj 			 решение 			 обеспечивает
NOUN 	 nmod 			 задач 			 решение
NOUN 	 nmod 			 формирования 			 задач
NOUN 	 nmod 			 системы 			 формирования
ADJ 	 amod 			 образовательных 			 услуг
NOUN 	 nmod 			 услуг 			 системы
NOUN 	 nmod 			 региона 			 системы
NOUN 	 nmod 			 округа 			 системы
NOUN 	 nmod 			 района 			 системы
ADP 	 appos 			 т 			 системы
X 	 fixed 			 е 			 т
NOUN 	 nmod 			 задач 			 решение
VERB 	 nmod 			 требующих 			 формирования
ADP 	 case 			 при 			 решении
NOUN 	 obl 			 решении 			 обеспечивает
ADJ 	 amod 			 географической 			 привязки
NOUN 	 nmod 			 привязки 			 решении
ADP 	 case 			 к 			 местности
NOUN 	 nmod 			 местности 			 привязки
PUNCT 	 punct 			 . 			 обеспечивает

-----------------------------------------------------------------

VERB 	 amod 			 разрабатывать 			 система 
NOUN 	 nsubj 			 система 			 обеспечивает 
VERB 	 ROOT 			 обеспечивать 			 обеспечивает 
ADP 	 case 			 к 			 примеру 
NOUN 	 obl 			 пример 			 обеспечивает 
NOUN 	 obj 			 решение 			 обеспечивает 
NOUN 	 nmod 			 задача 			 решение 
NOUN 	 nmod 			 формирование 			 задач 
NOUN 	 nmod 			 система 			 формирования 
ADJ 	 amod 			 образовательный 			 услуг 
NOUN 	 nmod 			 услуга 			 системы 
NOUN 	 nmod 			 регион 			 системы 
NOUN 	 nmod 			 округ 			 формирования 
NOUN 	 nmod 			 район 			 округа 
PROPN 	 appos 			 т 			 района 
NOUN 	 fixed 			 е 			 т 
NOUN 	 nmod 			 задача 			 формирования 
VERB 	 nmod 			 требовать 			 задач 
ADP 	 case 			 при 			 решении 
NOUN 	 obl 			 решение 			 требующих 
ADJ 	 amod 			 географический 			 привязки 
NOUN 	 nmod 			 привязка 			 решении 
ADP 	 case 			 к 			 местности 
NOUN 	 nmod 			 местность 			 привязки 
PUNCT 	 punct 			 . 			 обеспечивает 

-----------------------------------------------------------------

VERB 	 amod 			 разрабатывать 			 система
NOUN 	 nsubj 			 система 			 обеспечивает
VERB 	 root 			 обеспечивать 			 местности
ADP 	 case 			 к 			 примеру
NOUN 	 obl 			 пример 			 обеспечивает
NOUN 	 obj 			 решение 			 обеспечивает
NOUN 	 nmod 			 задача 			 решение
NOUN 	 nmod 			 формирование 			 задач
NOUN 	 nmod 			 система 			 формирования
ADJ 	 amod 			 образовательный 			 услуг
NOUN 	 nmod 			 услуга 			 системы
NOUN 	 nmod 			 регион 			 системы
NOUN 	 nmod 			 округ 			 региона
NOUN 	 nmod 			 район 			 округа
NOUN 	 nmod 			 тонна 			 района
CCONJ 	 case 			 е 			 задач
NOUN 	 nmod 			 задача 			 т
VERB 	 acl 			 требовать 			 задач
ADP 	 case 			 при 			 решении
NOUN 	 obl 			 решение 			 требующих
ADJ 	 amod 			 географический 			 привязки
NOUN 	 nmod 			 привязка 			 решении
ADP 	 case 			 к 			 местности
NOUN 	 nmod 			 местность 			 привязки